In [6]:
# Setup các kiểu
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
from datetime import datetime
import csv

# Nhớ sửa mấy link đến folder và file 
folder_path = r"D:\IE104.P11.CNVN\truyen"   
output_excel_path = r'hako_dataset.xlsx'
output_csv_path = r'hako_dataset.csv'

# Nhớ chỉnh sửa max_files ở dưới trước khi trích xuất và lưu

In [7]:
# Các hàm xử lý phụ
# Hàm xử lý ngày tháng năm
def parse_date(date_text):
    try:
        date_obj = datetime.strptime(date_text, '%d/%m/%Y')
        return date_obj.day, date_obj.month, date_obj.year
    except ValueError:
        print(f"Unable to parse date: {date_text}")
        return None, None, None
    
# Hàm để xử lý giá trị rỗng
def process_value(value, is_numeric=False):
    if value in [None, 'nan', 'NaN', 'N/A', '']:
        return 0 if is_numeric else "NOT FOUND"
    return value    

# Hàm chuyển đổi giá trị thành số
def convert_to_number(value):
    if isinstance(value, (int, float)):
        return value
    if isinstance(value, str):
        value = ''.join(filter(str.isdigit, value))
        return int(value) if value else 0
    return 0

# Hàm đếm số file html trong folder (có cải tiến lại vì trong folder có thể có folder con)
def count_html_files(folder_path):
    html_count = 0
    for filenames in os.listdir(folder_path):
        if filenames.endswith('.html'):
                html_count += 1
    return html_count

total = count_html_files(folder_path)   # Đếm số file html trong folder
print(f"Tổng số file html trong folder là: {total}")

Tổng số file html trong folder là: 177


In [8]:
# Các hàm trích xuất thông tin (muốn bỏ thông tin gì thì đóng comment đoạn code trích xuất thông tin đó)

# Trích xuất ID từ tên file
def extract_id_from_filename(filename):
    match = re.match(r'^(\\d+)', filename)
    return match.group(1) if match else None

# Trích xuất thông tin từ file HTML
def extract_info_from_html(file_path):
    try:
        file_id = extract_id_from_filename(os.path.basename(file_path))
        with open(file_path, 'r', encoding='utf-8') as file:
            # Trích xuất tên
            soup = BeautifulSoup(file, 'html.parser')
            title = soup.title.string.strip() if soup.title else "NOT FOUND"
            title = re.sub(r' - Cổng Light Novel - Đọc Light Novel$', '', title)
            
            # Trích xuất link
            canonical_link = soup.find('link', rel='canonical')
            canonical_url = canonical_link['href'] if canonical_link else "NOT FOUND"
            canonical_url = canonical_url.replace('docln.net', 'ln.hako.vn')
            
            # Trích xuất phương thức dịch
            method = "NOT FOUND"
            method_span = soup.find('div', class_='series-type')
            method_link = None
            if method_span:
                method_link = method_span.find('span')
            if method_link and method_link.string:
                method = method_link.string.strip()
            
            # Trích xuất thông tin về thể loại
            genres = []
            manga = "Not sure"
            anime = "Not sure"
            cd = "Not sure"
            origin = "japanese"
            genre_items = soup.find_all(class_='series-gerne-item')
            for item in genre_items:
                genre_text = item.get_text(strip=True)
                if "Manga" in genre_text: manga = "Yes"
                if "Anime" in genre_text: anime = "Yes"
                if "CD" in genre_text: cd = "Yes"
                if "Chinese" in genre_text: origin = "chinese"
                if "English" in genre_text: origin = "english"
                if "Korean" in genre_text: origin = "korean"
                elif not any(word in genre_text for word in ["Manga", "Anime", "CD", "Chinese", "English", "Korean"]):
                    genres.append(genre_text)
            
            # Trích xuất link ảnh
            image_link = "NOT FOUND"
            content_div = soup.find('div', class_='content img-in-ratio')
            if content_div and 'style' in content_div.attrs:
                style = content_div['style']
                match = re.search(r"url\\('([^']+)'\\)", style)
                if match:
                    image_link = match.group(1)
            
            # Trích xuất tác giả
            author = "NOT FOUND"
            author_span = soup.find('span', class_='info-name', string='Tác giả:')
            if author_span:
                info_value_span = author_span.find_next_sibling('span', class_='info-value')
                if info_value_span:
                    author_link = info_value_span.find('a')
                if author_link:
                    author = author_link.string.strip()
            
            # Trích xuất họa sĩ
            artist = "NOT FOUND"
            artist_span = soup.find('span', class_='info-name', string='Họa sĩ:')
            if artist_span:
                info_value_span = artist_span.find_next_sibling('span', class_='info-value')
                if info_value_span and info_value_span.string:
                    artist = info_value_span.string.strip()
            
            # Trích xuất kiểu trình bày
            showtype = 'light novel' if artist.lower() not in ['NOT FOUND', 'N/A'] else 'web novel'
            
            # Trích xuất tình trạng
            state = "NOT FOUND"
            state_span = soup.find('span', class_='info-name', string='Tình trạng:')
            if state_span:
                info_value_span = state_span.find_next_sibling('span', class_='info-value')
                if info_value_span:
                    state_link = info_value_span.find('a')
                    if state_link:
                        state = state_link.string.strip()
            
            # Trích xuất số like
            like = 0
            like_span = soup.find('span', class_='block feature-value')
            if like_span:
                like_link = like_span.find_next_sibling('span', class_='block feature-name')
                if like_link and like_link.string:
                    like_text = like_link.string.strip()
                    try:
                        like = float(like_text)
                    except ValueError:
                        pass
            
            # Trích xuất số từ
            nword = 0
            nword_span = soup.find('div', class_='statistic-name', string='Số từ')
            if nword_span:
                nword_link = nword_span.find_next_sibling('div', class_='statistic-value')
                if nword_link:
                    nword = nword_link.string.strip()
            
            # Trích xuất số lượt đánh giá
            # rate = 0
            # rate_span = soup.find('div', class_='statistic-name', string='Đánh giá')
            # if rate_span:
            #     rate_link = rate_span.find_next_sibling('div', class_='statistic-value')
            #     if rate_link and rate_link.string:
            #         rate_text = rate_link.string.strip()
            #         try:
            #             rate = float(rate_text)
            #         except ValueError:
            #             pass
            
            # Trích xuất số lượt xem
            # view = 0
            # view_span = soup.find('div', class_='statistic-name', string='Lượt xem')
            # if view_span:
            #     view_link = view_span.find_next_sibling('div', class_='statistic-value')
            #     if view_link:
            #         view = view_link.string.strip()
            
            # Trích xuất số lượt bình luận
            # ncom = 0
            # ncom_span = soup.find('span', class_='comments-count')
            # if ncom_span:
            #     ncom = ncom_span.string.strip()
            #     ncom = re.sub(r'[()]', '', ncom)
            
            # Trích xuất tên gọi khác
            fname = None
            fname_span = soup.find('div', class_='fact-value')
            if fname_span:
                fname_links = fname_span.find_all('div', class_='block pad-bottom-5')
                if fname_links:
                    fname_list = [link.get_text(strip=True) for link in fname_links if link.get_text(strip=True)]
                    fname = chr(10).join(fname_list) if fname_list else 'None'

            # Trích xuất người dịch
            trans = None
            id_o_l = None
            id_o = None
            trans_span = soup.find('span', class_='series-owner_name')
            if trans_span:
                trans = trans_span.string.strip()
                next_o = trans_span.find_next('a')
                if next_o and 'href' in next_o.attrs:
                    id_o_l = next_o['href']
                    if not id_o_l.startswith(('https://', 'http://')):
                        id_o_l = 'https://ln.hako.vn' + id_o_l
                    else:
                        id_o_l = id_o_l.replace('docln.net', 'ln.hako.vn')
                        id_o = re.search(r'/(\\d+)$', id_o_l)
                        id_o = id_o.group(1) if id_o else None

            # Trích xuất nhóm dịch
            team = None
            id_t_l = None
            id_t = None
            team_span = soup.find('div', class_='fantrans-value')
            if team_span:
                team = team_span.string.strip()
                next_t = team_span.find_next('a')
                if next_t and 'href' in next_t.attrs:
                    id_t_l = next_t['href']
                    if not id_t_l.startswith(('https://', 'http://')):
                        id_t_l = 'https://ln.hako.vn' + id_t_l
                    else:
                        id_t_l = id_t_l.replace('docln.net', 'ln.hako.vn')
                    id_t = re.search(r'/nhom-dich/(\\d+)', id_t_l)
                    id_t = id_t.group(1) if id_t else None

            # Trích xuất người tham gia
            atb = None
            id_j_l = None
            id_j = None
            atb_span = soup.find('div', class_='series-owner_share')
            if atb_span:
                atb_links = atb_span.find_all('a', class_='ln_info-name')
                if atb_links:
                    atb_list = [link.get_text(strip=True) for link in atb_links if link.get_text(strip=True)]
                    atb = chr(10).join(atb_list) if atb_list else 'None'
                    id_j_l_list = [link.get('href') for link in atb_links if link.get('href')]
                    id_j_l_list = ['https://ln.hako.vn' + url if not url.startswith(('https://', 'http://')) else url.replace('docln.net', 'ln.hako.vn') for url in id_j_l_list]
                    id_j_l = chr(10).join(id_j_l_list) if id_j_l_list else 'None'
                    id_j_list = [re.search(r'/(\\d+)$', url).group(1) for url in id_j_l_list if re.search(r'/(\\d+)$', url)]
                    id_j = chr(10).join(id_j_list) if id_j_list else 'None'

            # Trích xuất số tập
            def count_vol(soup):
                vol_spans = soup.find_all('span', class_='list_vol-title')
                return len(vol_spans)
            nvol = count_vol(soup)

            # Trích xuất số chương
            def count_chap(soup):
                chap_spans = soup.find_all('div', class_='chapter-name')
                return len(chap_spans)
            nchap = count_chap(soup)

            # Trích xuất thời gian bắt đầu và thời gian cập nhật mới nhất
            first_day, first_month, first_year = None, None, None
            latest_day, latest_month, latest_year = None, None, None
            chapter_time_divs = soup.find_all('div', class_='chapter-time')
            if chapter_time_divs:
                first_date_text = chapter_time_divs[0].get_text(strip=True)
                latest_date_text = chapter_time_divs[-1].get_text(strip=True)
                first_day, first_month, first_year = parse_date(first_date_text)
                latest_day, latest_month, latest_year = parse_date(latest_date_text)

            return (file_id, title, canonical_url, method, genres, manga, anime, cd, origin, image_link,
                    author, artist, showtype, state, like, nword,  fname, id_o, 
                    trans, id_o_l, id_t, team, id_t_l, id_j, atb, id_j_l, nvol, nchap, 
                    first_day, first_month, first_year, latest_day, latest_month, latest_year)
            #Thêm rate, view, ncom nếu cần
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ {file_path}: {str(e)}")
        return None

In [9]:
# Hàm xử lý folder
def process_folder(folder_path, max_files):
    all_data = []
    count = 0

    for filename in os.listdir(folder_path):
        if filename.endswith('.html'):
            file_path = os.path.join(folder_path, filename)
            try:
                # Trích xuất thông tin từ file HTML
                info = extract_info_from_html(file_path)
                # Kiểm tra xem info có phải là None không
                if info is None:
                    print(f"Không thể trích xuất thông tin từ file: {filename}")
                    continue

                # Giải nén thông tin
                (file_id, title, canonical_url, method, genres, manga, anime, cd, origin, image_link,
                 author, artist, showtype, state, like, nword, fname, id_o, 
                 trans, id_o_l, id_t, team, id_t_l, id_j, atb, id_j_l, nvol, nchap, 
                 first_day, first_month, first_year, latest_day, latest_month, latest_year) = info
                
                # Lưu thông tin vào DataFrame
                all_data.append({
                    'ID': process_value(file_id),
                    'Tựa đề': process_value(title),
                    'Link hako': process_value(canonical_url),
                    'Phương thức dịch': process_value(method),
                    'Thể loại': process_value(genres),
                    'Manga': process_value(manga),
                    'Anime': process_value(anime),
                    'CD': process_value(cd),
                    'Ngôn ngữ gốc': process_value(origin),
                    'Link ảnh': process_value(image_link),
                    'Tác giả': process_value(author),
                    'Họa sĩ': process_value(artist),
                    'Kiểu trình bày': process_value(showtype),
                    'Tình trạng': process_value(state),
                    'Số like': convert_to_number(process_value(like, is_numeric=True)),
                    'Số từ': convert_to_number(process_value(nword, is_numeric=True)),
                    # 'Số lượt đánh giá': convert_to_number(process_value(rate, is_numeric=True)),
                    # 'Số lượt xem': convert_to_number(process_value(view, is_numeric=True)),
                    # 'Số lượt bình luận': convert_to_number(process_value(ncom, is_numeric=True)),
                    'Fname': process_value(fname),
                    'ID người dịch': process_value(id_o),
                    'Người dịch': process_value(trans),
                    'Link người dịch': process_value(id_o_l),
                    'ID nhóm dịch': process_value(id_t),
                    'Nhóm dịch': process_value(team),
                    'Link nhóm dịch': process_value(id_t_l),
                    'ID người tham gia': process_value(id_j),
                    'Người tham gia': process_value(atb),
                    'Link người tham gia': process_value(id_j_l),
                    'Số tập': process_value(nvol),
                    'Số chương': process_value(nchap),
                    'Ngày bắt đầu': process_value(first_day),
                    'Tháng bắt đầu': process_value(first_month),   
                    'Năm bắt đầu': process_value(first_year),
                    'Ngày cập nhật cuối': process_value(latest_day),
                    'Tháng cập nhật cuối': process_value(latest_month),
                    'Năm cập nhật cuối': process_value(latest_year)
                })

                count += 1
                if count >= max_files:
                    break
            except Exception as e:
                print(f"Lỗi khi xử lý file {filename}: {str(e)}")

    print(f"Đã xử lý {count} file")
    return all_data



In [10]:
# Hàm lưu thông tin vào file excel
def save_to_excel(data, output_excel_path):
    df = pd.DataFrame(data)
    df.to_excel(output_excel_path, index=False, engine='openpyxl')
    print(f"Đã lưu thông tin vào file excel: {output_excel_path}")

# Hàm lưu thông tin vào file csv
def save_to_csv(data, output_csv_path):
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = data[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)
    print(f"Đã lưu thông tin vào file csv: {output_csv_path}")

In [11]:
# Chỉnh số lượng file muốn trích xuất (trích toàn bộ thì max_files = total)
max_files = 10 # Chỉnh số ở đây
dataset = process_folder(folder_path, max_files)

Đã xử lý 10 file


In [12]:
dataset

[{'ID': 'NOT FOUND',
  'Tựa đề': 'Seirei Tsukai no Blade Dance',
  'Link hako': 'https://ln.hako.vn/truyen/1-seirei-tsukai-no-blade-dance',
  'Phương thức dịch': 'Truyện dịch',
  'Thể loại': ['Action',
   'Adventure',
   'Comedy',
   'Fantasy',
   'Romance',
   'Ecchi',
   'Harem',
   'School Life',
   'Super Power',
   ''],
  'Manga': 'Yes',
  'Anime': 'Yes',
  'CD': 'Yes',
  'Ngôn ngữ gốc': 'japanese',
  'Link ảnh': 'NOT FOUND',
  'Tác giả': 'Shimizu Yuu',
  'Họa sĩ': 'Sakura Hanpen, Nimura Yuuji, Shimesada Kohada',
  'Kiểu trình bày': 'light novel',
  'Tình trạng': 'Đã hoàn thành',
  'Số like': 2944.0,
  'Số từ': 1071757,
  'Fname': 'NOT FOUND',
  'ID người dịch': 'NOT FOUND',
  'Người dịch': 'medassin',
  'Link người dịch': 'https://ln.hako.vn/thanh-vien/39',
  'ID nhóm dịch': 'NOT FOUND',
  'Nhóm dịch': 'Sonata no Koe',
  'Link nhóm dịch': 'https://ln.hako.vn/nhom-dich/3-sonata-no-koe',
  'ID người tham gia': 'NOT FOUND',
  'Người tham gia': 'NOT FOUND',
  'Link người tham gia': '

In [13]:
# Lưu vào file excel
save_to_excel(dataset, output_excel_path)

Đã lưu thông tin vào file excel: hako_dataset.xlsx


In [14]:
# Luu vào file csv
save_to_csv(dataset, output_csv_path)

Đã lưu thông tin vào file csv: hako_dataset.csv


In [15]:
# Muốn check gì thì check (nhớ sửa link)
df_excel = pd.read_excel(r"Hải Trần's folder\extract + save data\hako_dataset.xlsx")
df_csv = pd.read_csv(r"Hải Trần's folder\extract + save data\hako_dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: "Hải Trần's folder\\extract + save data\\hako_dataset.xlsx"